# Build a linear regression model on whoop data to predict next day's recovery based on previous day metrics

### target performance / metric: make recovery score predictions within 10% (lower or higher) than ground truth recovery score

165 days of data  
8 features  
1 target variable: recovery score  


In [1]:
#import linear regression:
#need hands-on ML book

SyntaxError: invalid syntax. Perhaps you forgot a comma? (555495630.py, line 1)